In [1]:
import pandas as pd
import numpy as np
import joblib 
import nltk
import re
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from pprint import pprint

In [2]:
!ls /kaggle/input/news-articles-with-ethnic-words

ethnic_dataset.joblib


In [4]:
ethnic_dataset = joblib.load("/kaggle/input/news-articles-with-ethnic-words/ethnic_dataset.joblib")

--2024-03-26 13:14:45--  https://github.com/aljubaer/Topic-Modeling-in-Bengali/blob/master/code/stop_words.txt
Resolving github.com (github.com)... 140.82.121.4
Connecting to github.com (github.com)|140.82.121.4|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: 'stop_words.txt'

stop_words.txt          [ <=>                ] 152.84K  --.-KB/s    in 0.03s   

2024-03-26 13:14:46 (5.02 MB/s) - 'stop_words.txt' saved [156513]

stop_words.txt


In [6]:
ethnic_tribe_names = [
    "চাকমা", "মারমা", "সাঁওতাল", "ত্রিপুরা", "গারো", "ওঁরাও", "তঞ্চ্যঙ্গা", "ম্রো", 
    "পাংখো", "চাক", "খেয়াং", "খুমি", "লুসাই","কুকি", "রাখাইন", "মণিপুরী",
    "হাজং", "খাসিয়া", "মং", "বর্মন", "পাহাড়ি", "মালপাহাড়ি", "মুন্ডা", "ভূমিজ",
    "কন্দ", "পাঙন", "লাওরা", "মুরং", "বাগদী"
] #"বম","কোচ","ডালু","কোল", "রাজবংশী", "পাত্র", "ভিল", "গণ্ড", "খাসি"

ethnicity_directed_words = [
    "আদিবাসী" , "আদিবাসি" , "উপজাতি", "নৃগোষ্ঠী"
]
ethnic_dictionary = ethnic_tribe_names + ethnicity_directed_words

In [7]:
'''
relevant_articles = []
THRESHOLD = 5
list_count = 0
for article in ethnic_dataset:
    if isinstance(article, str):
        ethnic_word_count = sum([article.split().count(ethnic_word) for ethnic_word in ethnic_dictionary])
        if ethnic_word_count > THRESHOLD:
            relevant_articles.append(article)
    else:
        list_count +=1 
print(f"Total Ethnic Article: {len(ethnic_dataset)}\nRelevant Ethnic Article: {len(relevant_articles)}")
'''

Total Ethnic Article: 456763
Relevant Ethnic Article: 34081


In [8]:
#list_count

0

In [11]:
'''
from IPython.display import FileLink

joblib.dump(relevant_articles, "relevant_articles.joblib")
FileLink(r'relevant_articles.joblib')
'''

/kaggle/working/relevant_articles.joblib

### Loading presaved data

In [5]:
import joblib
relevant_articles = joblib.load("/kaggle/input/news-articles-with-ethnic-words/relevant_articles.joblib")
len(relevant_articles)

34081

In [9]:
def valid_bengali_letters(char):
    return ord(char) >= 2433 and ord(char) <= 2543 

def get_replacement(char):
    if valid_bengali_letters(char):
        return char
    newlines = [10, 2404, 2405, 2551, 9576]
    if ord(char) in newlines: 
        return ' '
    return ' ';

def get_valid_lines(line):
    copy_line = ''
    for letter in line:
        copy_line += get_replacement(letter)
    return copy_line
def sent_to_words(sentences):
    for sentence in sentences:
        yield(nltk.word_tokenize(get_valid_lines(sentence)))  # deacc=True removes punctuations


def remove_stop_words(pre_data):
    f = open('stop_words.txt', "r+", encoding = 'utf-8')
    aa = f.read()
    data_stop = [d.strip() for d in aa.split()]
    new_data = []
    for i,line in enumerate(pre_data):
        i_new_data = []
        for word in line:
            if word not in data_stop and len(word) > 2:
                i_new_data.append(word)
        new_data.append(i_new_data)
    return new_data



skipped_data = []
def preprocess_corpus(data):
    if not isinstance(data, list):
        raise TypeError("Data must be a list of strings")

    # Remove new line characters and distracting single quotes
    cleaned_data = []
    for sent in data:
        if isinstance(sent, str):
            cleaned_sent = re.sub('\s+', ' ', sent)
            cleaned_sent = re.sub("\'", "", cleaned_sent)
            cleaned_data.append(cleaned_sent)
        else:
            skipped_data.append(sent)

    data = cleaned_data

    data_words = list(sent_to_words(data))

    for row, single_data in enumerate(data_words):
        for column, word in enumerate(single_data):
            data_words[row][column] = word # stem

    # Build the bigram and trigram models
    bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
    trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

    # Faster way to get a sentence clubbed as a trigram/bigram
    bigram_mod = gensim.models.phrases.Phraser(bigram)
    trigram_mod = gensim.models.phrases.Phraser(trigram)

    data_ready = remove_stop_words(data_words)
    return data_ready

In [10]:
!wget https://github.com/aljubaer/Topic-Modeling-in-Bengali/blob/master/code/stop_words.txt
!ls

--2024-03-26 16:24:33--  https://github.com/aljubaer/Topic-Modeling-in-Bengali/blob/master/code/stop_words.txt
Resolving github.com (github.com)... 140.82.116.3
Connecting to github.com (github.com)|140.82.116.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: 'stop_words.txt'

stop_words.txt          [ <=>                ] 152.84K  --.-KB/s    in 0.03s   

2024-03-26 16:24:33 (4.79 MB/s) - 'stop_words.txt' saved [156506]

stop_words.txt


In [12]:
import re
data = relevant_articles
data_ready = preprocess_corpus(data)

id2word = corpora.Dictionary(data_ready)
corpus = [id2word.doc2bow(text) for text in data_ready]
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=30,
                                           passes=30,
                                           alpha='auto',
                                           iterations=1000,
                                           per_word_topics=True)

pprint(lda_model.print_topics(num_words = 25))
joblib.dump(lda_model, "topic_model.joblib")


NameError: name 'nltk' is not defined

In [ ]:
from IPython.display import FileLink
FileLink(r'topic_model.joblib')

### Inspect the dataset
 - If `চাকমা`, `ত্রিপুরা` is in someone's name
     - `কোচ` is a tribe name. So many of our articles are on sports.
     -  We are removing `"বম","কোচ","ডালু","কোল", "রাজবংশী", "পাত্র", "ভিল", "গণ্ড", "খাসি"` from our list of ethnic words since they may mean many things aside from a tribe name.
 - Removing `মুসলিম` from `পাঙন মুসলিম` since it is picking religious articles.
 - We have this noise everywhere.`\n\n\xa0\n\n` , `\u200c্`
 - There are some arrays. We need to parse them.
### Questions
 - Why are there`রোহিঙ্গা` articles in our ethnic dataset?
 - How do we determine which articles are actually on ethnic people? In many articles, the perpetrators of a crime just happen to be a `চাকমা`. Those articles are not about ethnic people!

In [ ]:
ethnic_dataset[30109]